<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

def local_training(model, data, target, optimizer, epochs=1):
    model.train()
    for _ in range(epochs):
        optimizer.zero_grad()
        output = model(data)
        loss = ((output - target) ** 2).mean()
        loss.backward()
        optimizer.step()
    return model.state_dict()

# Simulate multiple clients
num_clients = 5
models = [SimpleModel() for _ in range(num_clients)]
optimizers = [optim.SGD(model.parameters(), lr=0.01) for model in models]
data = torch.randn(10, 10)  # Simulated data
target = torch.randn(10, 1)  # Simulated target

for round in range(3):  # 3 rounds of federated learning
    local_weights = []
    for i in range(num_clients):
        local_weights.append(local_training(models[i], data, target, optimizers[i]))
    # Aggregating weights (simple average)
    global_weights = {k: sum([w[k] for w in local_weights]) / num_clients for k in local_weights[0]}
    for model in models:
        model.load_state_dict(global_weights)
    print(f'Round {round + 1} completed')